In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')
test.head()

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


In [3]:
val = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
val.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [4]:
sub = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
sub.head()

,comment_id,score
0,114890,0.5
1,732895,0.5
2,1139051,0.5
3,1434512,0.5
4,2084821,0.5


In [5]:
toxic_train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
toxic_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
toxic_train['y'] = (toxic_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis = 1) >0 ).astype(int)
toxic_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,y
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [7]:
toxic_train['y'].value_counts()

0    143346
1     16225
Name: y, dtype: int64

In [8]:
toxic_train_undersample = toxic_train[toxic_train['y'] == 0].sample(n = 16225, random_state = 2020)
toxic = pd.concat([toxic_train_undersample, toxic_train[toxic_train['y'] == 1]])
toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,y
69816,bac4e5358a94395c,"""\n\nI have never used Wikipedia for editing a...",0,0,0,0,0,0,0
105772,35eb8cb052edd783,Vandalsim warning \n\nhttp://en.wikipedia.org/...,0,0,0,0,0,0,0
51494,89c33ba9c1cc1e28,Dog i.e. Renee question \n\nDid anyone from t...,0,0,0,0,0,0,0
27518,48d287d6379a207e,"""\n\n DistroWatch != popularity \n\nI've rever...",0,0,0,0,0,0,0
155259,bb3bfb52f2e08419,I like coulises. They are often very tasty. My...,0,0,0,0,0,0,0


In [9]:
toxic = toxic.drop(['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis = 1)
toxic.head()

,comment_text,y
69816,"""\n\nI have never used Wikipedia for editing a...",0
105772,Vandalsim warning \n\nhttp://en.wikipedia.org/...,0
51494,Dog i.e. Renee question \n\nDid anyone from t...,0
27518,"""\n\n DistroWatch != popularity \n\nI've rever...",0
155259,I like coulises. They are often very tasty. My...,0


In [10]:
toxic['y'].value_counts()

0    16225
1    16225
Name: y, dtype: int64

In [11]:
toxic['comment_text'].apply(lambda x : len(str(x).split())).max()

1411

In [12]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(toxic['comment_text'])
comment_seq = t.texts_to_sequences(toxic['comment_text'])
len(t.word_index)

69947

In [13]:
from keras.preprocessing import sequence

comment_pad = sequence.pad_sequences(comment_seq, maxlen = 1411)
target = toxic['y']

In [14]:
import sklearn
comment_pad, target = sklearn.utils.shuffle(comment_pad, target, random_state = 2021)

In [15]:
import os, gc

del comment_seq, toxic, toxic_train_undersample, toxic_train
gc.collect()

21

In [16]:
embedding_words = {}
file = open('../input/glove840b300dtxt/glove.840B.300d.txt', 'r', encoding = 'utf-8')
for line in file:
    values = line.split(' ')
    word = values[0]
    vector = np.asarray([float(val) for val in values[1:]])
    embedding_words[word] = vector
file.close()

In [17]:
embedding_matrix = np.zeros((len(t.word_index) + 1, 300))
for word,val in t.word_index.items() :
    vector = embedding_words.get(word)
    if vector is not None:
        embedding_matrix[val] = vector

In [18]:
def model():
    model = keras.Sequential()
    model.add(keras.layers.Embedding(len(t.word_index) + 1, output_dim = 300, weights = [embedding_matrix], input_length = 1411, trainable = False ))
    model.add(keras.layers.LSTM(33, dropout = 0.5, return_sequences=True))
    model.add(keras.layers.LSTM(10, dropout = 0.3))
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    return model

model = model()

2021-12-23 13:15:10.705124: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 13:15:10.838374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 13:15:10.839417: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-23 13:15:10.840999: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [19]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1411, 300)         20984400  
_________________________________________________________________
lstm (LSTM)                  (None, 1411, 33)          44088     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                1760      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 21,030,259
Trainable params: 45,859
Non-trainable params: 20,984,400
_________________________________________________________________


In [20]:
history = model.fit(comment_pad, target, epochs = 8, batch_size = 128)

2021-12-23 13:15:14.991955: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 183147800 exceeds 10% of free system memory.
2021-12-23 13:15:15.203632: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8


2021-12-23 13:15:20.107568: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


254/254 [==============================] - 42s 142ms/step - loss: 0.3555 - accuracy: 0.8513
Epoch 2/8
254/254 [==============================] - 36s 143ms/step - loss: 0.2542 - accuracy: 0.8967
Epoch 3/8
254/254 [==============================] - 36s 143ms/step - loss: 0.2249 - accuracy: 0.9088
Epoch 4/8
254/254 [==============================] - 37s 144ms/step - loss: 0.2400 - accuracy: 0.9045
Epoch 5/8
254/254 [==============================] - 36s 143ms/step - loss: 0.2329 - accuracy: 0.9089
Epoch 6/8
254/254 [==============================] - 37s 144ms/step - loss: 0.2080 - accuracy: 0.9165
Epoch 7/8
254/254 [==============================] - 36s 143ms/step - loss: 0.1993 - accuracy: 0.9216
Epoch 8/8
254/254 [==============================] - 37s 144ms/step - loss: 0.1951 - accuracy: 0.9225


In [21]:
comment_seq_test = t.texts_to_sequences(test['text'])
comment_pad_test = sequence.pad_sequences(comment_seq_test, maxlen = 1411)

In [22]:
y_test = model.predict(comment_pad_test)

2021-12-23 13:20:16.147908: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 42538828 exceeds 10% of free system memory.


In [23]:
sub['score'] = y_test
sub.head()

,comment_id,score
0,114890,0.006591
1,732895,0.431618
2,1139051,0.015597
3,1434512,0.003121
4,2084821,0.800016


In [24]:
sub[['comment_id', 'score']].to_csv("submission.csv", index=False)